Script for genenerate everything for dadda tree

In [65]:
import pprint
import numpy as np
import collections as co
import random  

For each stage from ,…,1, reduce each column starting at the lowest-weight column, c_{0} according to these rules:

In [66]:
# dadda tree needs a total of 8 stages
# a = '{:032b}'.format(bin(-1)) # 100
# b = '{:032b}'.format(bin(100)) # 100
# op_a = 1256
# op_b = 312
op_a = random.randint(0, 2**32)
op_b = random.randint(0, 2**32)
a = format(op_a & 0xFFFFFFFF, '032b')
b = format(op_b & 0xFFFFFFFF, '032b')
# a = format(bin(-1), "32b")
# b = format(bin(100), "32b")
a = a[::-1] # multiplicant, top 
b = b[::-1] # multiplier, bottom
py_output_dir = "tmp"


print("left to right: low to high")
print("A: ", a)
print("B: ", b)

# negative is bit strange







left to right: low to high
A:  01001001100111000111100010000101
B:  01110111011110110111000101011010


In [67]:
f4 = open(f"{py_output_dir}/dadda_tree_o.sv", "w")
# f4.write("module dadda_tree(\n")
# f4.write("  input logic[31:0] opA,      \n")
# f4.write("  input logic[31:0] opB,      \n")
# f4.write("  output logic[63:0] prodAB   \n")
# f4.write(");\n\n")
# f4.write("logic [31:0] comb[32]; \n")
# f4.write("genvar i, j; \n")
# f4.write("for(i = 0; i < 32; ++i) begin \n")
# f4.write("  for(j = 0; j < 32; ++j) begin \n")
# f4.write("    assign comb[i][j] = opB[i] & opA[j]; \n")
# f4.write("  end \n")
# f4.write("end \n\n")
f4.write("""module dadda_tree(
    input logic[31:0] opA,
    input logic[31:0] opB,
    output logic[63:0] row_top,
    output logic[63:0] row_bot
);

logic [31:0] comb[32];

genvar i, j;
for(i = 0; i < 32; ++i) begin
    for(j = 0; j < 32; ++j) begin
        assign comb[i][j] = opB[i] & opA[j];
    end
end
""")

##############################################
#
#   Partial Product
#
##############################################
f = open(f"{py_output_dir}/output", "w")



# long multiplication
empty = "       "
long_mul_np = np.full((32, 64), fill_value=2 ,dtype=np.int32)
long_mul = co.defaultdict(dict)
long_mul_cord = [[ empty for _ in range(64)] for _ in range(32)]

str_cord_to_tuple = dict()
# pprint.pprint(long_mul)
for i in range(0, 32): # total of 32 row
    num = int(b[i])    # get current multiplier bit
    for j in range(0, 32): # going through all multiplicand bits
        pp = int(a[j]) & num
        long_mul_np[i][63 - i - j] = pp
        long_mul[f"p{i:02d}{j:02d}"] = pp
        str_cord_to_tuple[f"p{i:02d}{j:02d}"] = (i, j) # A use j, B use i

        long_mul_cord[i][63 - i - j] = f"p{i:02d}{j:02d}"
        # f4.write(f"logic p{i:02d}{j:02d};\n")
        # f4.write(f"assign p{i:02d}{j:02d} = opB[{i}] & opA[{j}];\n")

    temp = [str(k) for k in long_mul_np[i]]
    # file portion ===========================
    f.write("".join(temp))
    f.write("\n")

f.write("\n")
for item in long_mul_cord:
    f.write("".join(item)) 
    f.write(f"        {len(item)}")
    f.write("\n")

f.write("\n")
f.close()


# pprint.pprint(long_mul)


In [68]:
# rotate pyramid
f2 = open(f"{py_output_dir}/output2.txt","w")


rot_pyra = [[] for _ in range(64)]
for i in range(63): # column 
    for j in range(32): # row
        if(long_mul_cord[j][63 - i] == empty): continue
        rot_pyra[i].append(long_mul_cord[j][63 - i]) 


# pprint.pprint(rot_pyra)
for item in rot_pyra:
    # f2.write("".join(str(i) for i in item))
    f2.write(" ".join(item))
    f2.write(f"      {len(item)}")
    f2.write("\n")
f2.close()

# print(rot_pyra)

In [69]:

# functions
def halfAdd(op1, op2):
    s = op1 ^ op2
    carry = op1 & op2
    return s, carry
    
def fullAdd(op1, op2, cin):
    s = op1 ^ op2 ^ cin
    carry = (op1 & op2) | (cin & (op1 ^ op2))
    return s, carry

# full_adder_sv_write
def fullAdd_sv(module_name, A, B, c_in, sum, c_out, file):
    module_str = f"FA {module_name}(.A_i({A}), .B_i({B}), .c_in({c_in}), .S_o({sum}), .c_out({c_out}));\n"
    file.write(module_str)
    return
    
# ha_adder_sv_write
def halfAdd_sv(module_name, A, B, sum, c_out, file):
    module_str = f"HA {module_name}(.A_i({A}), .B_i({B}), .S_o({sum}), .c_out({c_out}));\n"
    file.write(module_str)
    return




In [70]:
# the d_j we pick for 32 bit is going to be 28, total of 8 stages
d_j = 28    # goes from 28, 19, 13, 9, 6, 4, 3, 2
j = 8       # goes from  8,  7,  6, 5, 4, 3, 2, 1

stage_dict = [28, 19, 13, 9, 6, 4, 3, 2]

# print(fullAdd(1,1,1))
# print(halfAdd(1,1))
pending_table = [[] for _ in range(64)]
# print(pending_table)


f3 = open(f"{py_output_dir}/output_d_j_8.txt","w")

# print(len(rot_pyra))
for d_j in stage_dict:
    f3.write(f"stage {j} ======================================================================= \n")
    f4.write(f"// stage {j} begin ======================================================================================================= \n") 

    for i in range(0, 64):
        if( (len(rot_pyra[i]) + len(pending_table[i])) <= d_j): continue
        add_col_c = 0
        while( (len(rot_pyra[i]) + len(pending_table[i])) > d_j):
            if( (len(rot_pyra[i]) + len(pending_table[i])) == d_j + 1):
                ops1, ops2 = rot_pyra[i][0], rot_pyra[i][1] # get the strings
                op1, op2 = long_mul[ops1], long_mul[ops2]   # get the value

                rot_pyra[i] = rot_pyra[i][2:]               # pop the first two
                # rot_pyra[i].pop(0)                            
                # rot_pyra[i].pop(0)

                # two pop inject to front, top + 1 


                s, c = halfAdd(op1, op2)
                
                # logic [31:0] comb[32];
                # comb[i] determine the row, j determine the column
                # (i, j): A use j, B use i
                if(ops1 in str_cord_to_tuple): 
                    ops1 = str_cord_to_tuple[ops1]
                    ops1 = f"comb[{ops1[0]}][{ops1[1]}]"

                if(ops2 in str_cord_to_tuple):
                    ops2 = str_cord_to_tuple[ops2]
                    ops2 = f"comb[{ops2[0]}][{ops2[1]}]"
                    
                    
                
                # halfAdd_sv(f"HA_{ops1}_{ops2}" ,ops1, ops2, f"HS_{ops1}_{ops2}", f"HC_{ops1}_{ops2}", f4) # sv module instantiation

                f4.write(f"logic S_s{j}_{i}_{add_col_c}, C_s{j}_{i}_{add_col_c};\n")
                halfAdd_sv(f"HA_s{j}_{i}{add_col_c}" ,ops1, ops2, f"S_s{j}_{i}_{add_col_c}", f"C_s{j}_{i}_{add_col_c}", f4) # sv module instantiation new one
                # long_mul[f"HS_{ops1}_{ops2}"] = s           # sum
                # long_mul[f"HC_{ops1}_{ops2}"] = c           # carry 
                long_mul[f"S_s{j}_{i}_{add_col_c}"] = s           # sum
                long_mul[f"C_s{j}_{i}_{add_col_c}"] = c           # carry 

                # rot_pyra[i].append(f"HS_{ops1}_{ops2}")                    # by the rule, sum is placed at the bottom of current col
                # rot_pyra[i+1].append(f"HC_{ops1}_{ops2}")                  # by the rule, carry is placed at the bottom of next col
                
                # rot_pyra[i].append(f"S_s{j}_{i}_{add_col_c}")                    # by the rule, sum is placed at the bottom of current col
                # rot_pyra[i+1].append(f"C_s{j}_{i}_{add_col_c}")                  # by the rule, carry is placed at the bottom of next col
                # # rot_pyra[i].insert(0, f"S_s{j}_{i}{add_col_c}")                    # by the rule, sum is placed at the bottom of current col
                # rot_pyra[i+1].insert(0, f"C_s{j}_{i}{add_col_c}")                  # by the rule, carry is placed at the bottom of next col
                
                pending_table[i].append(f"S_s{j}_{i}_{add_col_c}")
                pending_table[i+1].append(f"C_s{j}_{i}_{add_col_c}")
                add_col_c += 1    
            else:

                op_s1, op_s2, cin_s = rot_pyra[i][0], rot_pyra[i][1], rot_pyra[i][2]    # get strings 
                op1, op2, cin = long_mul[op_s1], long_mul[op_s2], long_mul[cin_s]       # get values

                # rot_pyra[i].pop(0)
                # rot_pyra[i].pop(0)
                # rot_pyra[i].pop(0)
                rot_pyra[i] = rot_pyra[i][3:]

                s, c = fullAdd(op1, op2, cin)


                if(op_s1 in str_cord_to_tuple): 
                    op_s1 = str_cord_to_tuple[op_s1]
                    op_s1 = f"comb[{op_s1[0]}][{op_s1[1]}]"

                if(op_s2 in str_cord_to_tuple):
                    op_s2 = str_cord_to_tuple[op_s2]
                    op_s2 = f"comb[{op_s2[0]}][{op_s2[1]}]"

                if(cin_s in str_cord_to_tuple):
                    cin_s = str_cord_to_tuple[cin_s]
                    cin_s = f"comb[{cin_s[0]}][{cin_s[1]}]"

                f4.write(f"logic S_s{j}_{i}_{add_col_c}, C_s{j}_{i}_{add_col_c};\n")
                # fullAdd_sv(f"FA_{op_s1}_{op_s2}_{cin_s}" ,op_s1, op_s2, cin_s, f"FS_{op_s1}_{op_s2}_{cin_s}", f"FC_{op_s1}_{op_s2}_{cin_s}", f4) # sv module instantiation
                fullAdd_sv(f"FA_s{j}_{i}_{add_col_c}" ,op_s1, op_s2, cin_s, f"S_s{j}_{i}_{add_col_c}", f"C_s{j}_{i}_{add_col_c}", f4) # sv module instantiation new one

                # long_mul[f"FS_{op_s1}_{op_s2}_{cin_s}"] = s           # sum
                # long_mul[f"FC_{op_s1}_{op_s2}_{cin_s}"] = c           # carry 
                long_mul[f"S_s{j}_{i}_{add_col_c}"] = s           # sum
                long_mul[f"C_s{j}_{i}_{add_col_c}"] = c           # carry 
                
                # rot_pyra[i].append(f"FS_{op_s1}_{op_s2}_{cin_s}")                    # by the rule, sum is placed at the bottom of current col
                # rot_pyra[i+1].append(f"FC_{op_s1}_{op_s2}_{cin_s}")                  # by the rule, carry is placed at the bottom of next col
               
            
                # rot_pyra[i].append(f"S_s{j}_{i}_{add_col_c}")                    # by the rule, sum is placed at the bottom of current col
                # rot_pyra[i+1].append(f"C_s{j}_{i}_{add_col_c}")                  # by the rule, carry is placed at the bottom of next col
                # rot_pyra[i].insert(0, f"S_s{j}_{i}{add_col_c}")                    # by the rule, sum is placed at the bottom of current col
                # rot_pyra[i+1].insert(0, f"C_s{j}_{i}{add_col_c}")                  # by the rule, carry is placed at the bottom of next col
                
                pending_table[i].append(f"S_s{j}_{i}_{add_col_c}")
                pending_table[i+1].append(f"C_s{j}_{i}_{add_col_c}")
                add_col_c += 1    
    j -= 1  # level change
    

    for ti in range(64):
        # print(pending_table[ti])
        while pending_table[ti]:
            item = pending_table[ti].pop(0)
            rot_pyra[ti].insert(0, item)
    print("\n")
    
    # stage 8
    f4.write(f"// stage {j+1} end ======================================================================================================= \n\n")
    for item in rot_pyra:
        # f2.write("".join(str(i) for i in item))
        f3.write("   ".join(item))
        f3.write(f"      {len(item)}")
        f3.write("\n")

    f3.write("======================================================================= \n")


f3.close()

In [71]:
# sanity and write down the last add 

# the two rows we get from the reduction
top_row = []
bot_row = []
for i in range(len(rot_pyra)):
    top_temp = "1'b0"
    bot_temp = "1'b0"

    if(len(rot_pyra[i]) >= 1):  # top only exists if the length is at least 1
        top_temp = rot_pyra[i][0]

    if(len(rot_pyra[i]) >=2):   # bot only exists if the length is at least 2
        bot_temp = rot_pyra[i][1]
    
    if(top_temp in str_cord_to_tuple): # if that is part of partial product
        top_temp = str_cord_to_tuple[top_temp]
        top_temp = f"comb[{top_temp[0]}][{top_temp[1]}]"

    if(bot_temp in str_cord_to_tuple):  # if that is part of partial product
        bot_temp = str_cord_to_tuple[bot_temp]  
        bot_temp = f"comb[{bot_temp[0]}][{bot_temp[1]}]"

    top_row.insert(0, top_temp)
    bot_row.insert(0, bot_temp)

# stringify, comma separated
top_row = "{" + ", ".join(top_row) + "}"
bot_row = "{" + ", ".join(bot_row) + "}"

# naive implementation for dadda tree, there might be a better way to do add
f4.write(f"""
// logic [63:0] last_top_row, last_bot_row;
assign row_top = {top_row};
assign row_bot = {bot_row};
// assign prodAB = last_top_row + last_bot_row;    // combine and find the sum, naive implementation
""")
# print(top_row)
# print(bot_row)



f4.write("endmodule\n")
f4.close()

f5 = open(f"{py_output_dir}/last_set.txt", "w")
for i in range(64):
    for j in range(len(rot_pyra[i])):
        f5.write(rot_pyra[i][j] + " ")
        rot_pyra[i][j] = long_mul[rot_pyra[i][j]]
    f5.write("\n")

rot_pyra[0].append(0)

for item in rot_pyra:
    # f2.write("".join(str(i) for i in item))
    f5.write(" ".join([str(i) for i in item]))
    f5.write(f"      {len(item)}")
    f5.write("\n")
f5.close()

In [72]:
# sum combining (unsigned)

final_a = [ str(rot_pyra[i][0]) for i in range(62, -1, -1)]
final_b = [ str(rot_pyra[i][1]) for i in range(62, -1, -1)]

final_a_s = "0b"+"".join(final_a)
final_b_s = "0b"+"".join(final_b)
print(final_a_s)
print(final_b_s)

num_a = int(final_a_s, 2)
num_b = int(final_b_s, 2)



dadda = num_a + num_b
correct = op_a * op_b
print(op_a, op_b)
print("dadda: ", num_a+num_b)
print("multi: ", op_a * op_b)
print(dadda == correct)

print("dadda: ", hex(num_a+num_b))
print("dadda: ", bin(dadda))
print("multi: ", hex(op_a * op_b))
print("dadda: ", str(bin(dadda))[-32:-1])




0b011000011111101011101001011000011101110111111111101110110111100
0b000100000000001000101101001111001010100000000100100010000000000
2703112594 1519312622
dadda:  4106873082751361468
multi:  4106873082751361468
True
dadda:  0x38fe8b4f430221bc
dadda:  0b11100011111110100010110100111101000011000000100010000110111100
multi:  0x38fe8b4f430221bc
dadda:  0100001100000010001000011011110


In [ ]:


# records for the dadda tree sv 
# lower_partial_sum = {1'b0, row_top[31:0]} + {1'b0, row_bot[31:0]};          // lower half of the product
# lower_partial_carry = lower_partial_sum[32];                                // the carry from the lower part 
# upper_partial_sum = row_top[63:32] + row_bot[63:32] + lower_partial_carry;  // upper half of the product adds with part# 
# if(should_neg) begin
#     lower_partial_sum = {1'b0, ~row_top[31:0]} + {1'b0, ~row_bot[31:0]} + should_neg + 1'b1;         // negation, one complement + 1
#     lower_partial_carry = lower_partial_sum[32];                                        // the carry from the lower part 
#     upper_partial_sum = (~row_top[63:32]) + (~row_bot[63:32]) + lower_partial_carry;    // upper half of the product adds with partial
# end 

For synth different top level, edit synthesis.ctl

Change `set design_toplevel divider`  to  `set design_toplevel some_module_name_you_want`

For lint different top level, edit lint.ctl

Replace the module name of all the ones with `top` flag to `some_module_name_you_want`